In [11]:
# Atnaujintas variantas, po to, kaip padaryti pakeitimai failų varduose 2022 03 26
# Čia variantas su pacientų duomenų padalijimu į train ir test. 
#
# Skriptas zive_creat_beats_attrib skirtas zive EKG įrašuose esančių pūpsnių atributų masyvui all_beats_attr
# sukūrti, kad būtų galima juos panaudoti sekų formavimui:
# - atsikratome nepageidaujamų anotacijų: 'U'  ------- ///////////////////// pataisyti, neatsikratyti
# - iš anotacijų suformuojame klasių numerius
# - apskaičiuojami ir į atributus įrašomi RR intervalai: RRl, RRr. Atributų eilutėse,
#  atitinkančios pirmą ir paskutinį pūpsnį, RRl ir RRr reikšmės lygios
#  sentinel = -1  - nereikia, skaičiuojama programiškai /////////////// ///////////////////////

# Pacientų kodų sąrašas SubjCodes (userNr+file_name) paimamas iš info_create_z.json, kurį
# suformuoja zive_create_npy. Masyvas all_beats_attr įrašomas į failą all_beats_attr_z.csv.
# 
import pandas as pd
import numpy as np
from pathlib import Path
from icecream import ic
import json, time, sys

from zive_util_vu import runtime, read_rec_attrib, split_SubjCode, get_annotations_table, print_annotations_table


import warnings
warnings.filterwarnings("ignore")

my_os=sys.platform

print("OS in my system : ",my_os)

if my_os != 'linux':
    OS = 'Windows'
else:  
    OS = 'Ubuntu'

# Pasiruošimas

# //////////////// NURODOMI PARAMETRAI /////////////////////////////////////////////////////

# Bendras duomenų aplankas, kuriame patalpintas subfolderis name_db

if OS == 'Windows':
    Duomenu_aplankas = 'D:\DI\Data\MIT&ZIVE\VU'   # variantas: Windows
else:
    Duomenu_aplankas = '/home/kesju/DI/Data/MIT&ZIVE/VU'   # arba variantas: UBUNTU, be Docker

# jei variantas Docker pasirenkame:
# Duomenu_aplankas = '/Data/MIT&ZIVE'

#  MIT2ZIVE duomenų aplankas
db_folder = 'DUOM_VU'

# Aplankas su MIT2ZIVE EKG įrašais (.npa) ir anotacijomis (.json)
rec_folder = 'records_npy'
rec_list_npy = 'list_tst_npy.json'
train_beats_attr_fname ='train_beats_attr_z.csv'

# Kas kiek išvedamas apdorotų duomenų skaičius
show_period = 100

# Failai pūpsnių klasių formavimui
annot_grouping = {'N':'N', 'S':'S', 'V':'V', 'U':'U'}
selected_beats = {'N':0, 'S':1, 'V':2, 'U':3}

# ///////////////////////////////////////////////////////////////////////////////

# Nuoroda į MIT2ZIVE duomenų aplanką
db_path = Path(Duomenu_aplankas, db_folder)

# Nuoroda į aplanką su MIT2ZIVE EKG įrašais (.npa) ir anotacijomis (.json)
rec_dir = Path(db_path, rec_folder)


# Nuskaitome failą info_create.json ir duomenų rinkinio parametrus
file_path = Path(rec_dir,'info_create_z.json')
with open(file_path) as json_file:
    info_create = json.load(json_file)

fs = info_create['fs'] # diskretizavimo dažnumas
SubjCodes =  info_create['SubjCodes'] # pacientų įrašų sąrašas

# Sąrašas testavimui
SubjCodes = \
[10000, 10010, 10020, 10030, 10040, 10041, 10050, 10042, 10060, 10043, 10070, 10011,
 10080, 10031, 10090, 10100, 10110, 10021, 10120, 10022, 10130]

annot_list  = info_create['annot_list'] # anotacijų sąrašas
# print(annot_list)

# Susikuriame pagalbinį anotacijų žodyną - dictionary beats_annot
nr_sequence = list(range(13))
beats_annot = dict(zip(annot_list, nr_sequence))

print("\nSkriptas skirtas zive EKG įrašuose esančių pūpsnių atributų sarašui all_beats_attr sukurti")
print("\nBendras Zive duomenų aplankas: ", Duomenu_aplankas)
print("Zive EKG įrašų aplankas: ", rec_dir)
print("EKG įrašų atributų sąrašas:", rec_list_npy)

print("Diskretizavimo dažnis: ", fs)
print("\nPacientų įrašų kodų sąrašas:\n",SubjCodes)
print("\nAnotacijų sąrašas:\n", annot_list)


OS in my system :  win32

Skriptas skirtas zive EKG įrašuose esančių pūpsnių atributų sarašui all_beats_attr sukurti

Bendras Zive duomenų aplankas:  D:\DI\Data\MIT&ZIVE\VU
Zive EKG įrašų aplankas:  D:\DI\Data\MIT&ZIVE\VU\DUOM_VU\records_npy
EKG įrašų atributų sąrašas: list_tst_npy.json
Diskretizavimo dažnis:  200

Pacientų įrašų kodų sąrašas:
 [10000, 10010, 10020, 10030, 10040, 10041, 10050, 10042, 10060, 10043, 10070, 10011, 10080, 10031, 10090, 10100, 10110, 10021, 10120, 10022, 10130]

Anotacijų sąrašas:
 ['N', 'S', 'V', 'U']


In [12]:
# I-a dalis. Nustatomas anotacijų pasiskirstymas per visus įrašus.
# from zive_util_vu import get_SubjCode

pd.set_option("display.max_rows", 1000, "display.max_columns", 19)
pd.set_option('display.width', 1000)

# Atliekama pirminė EKG įrašų anotacijų analizė

# Pasiruošimas ciklui per pacientų įrašus
labels_rec_all = pd.DataFrame(columns=beats_annot.keys(),dtype=int)
labels_rec_all.insert(0,"SubjCode",0)
labels_rec = []

# Pacientų įrašų sąrašas bandymams (yra dar viena vieta):
# SubjCodes = ['100','101', '230', '231']

# Ciklas per pacientų įrašus

for SubjCode in SubjCodes:
    # Paciento anotacijų nuskaitymas, nustatomas anotacijų 
    # ir užduotų klasių pasiskirstymas per visus įrašus

    labels_rec = np.zeros(labels_rec_all.shape[1],dtype=object)
    labels_rec[0] = SubjCode

    # Nuskaitome anotacijas ir jų indeksus
    file_path = Path(rec_dir, str(SubjCode) + '.json')
    # print(file_path)

 #  load data using Python JSON module
    atr_sample, atr_symbol = read_rec_attrib(rec_dir, SubjCode)

    # Ciklas per visas paciento įrašo anotacijas (simbolius)
    for symbol in atr_symbol:
        # Gaunamas anotacijos simbolio numeris anotacijų sąraše
        label = beats_annot.get(symbol)
        if (label == None):
            continue
        labels_rec[label+1] +=1

    # Sumuojame į bendrą masyvą
    # print(labels_rec)
    labels_rec_all.loc[len(labels_rec_all)] = labels_rec

# Ciklo per pacientų įrašus pabaiga

# Parodomas anotacijų ir užduotų klasių pasiskirstymas per visus įrašus
print("\nAnotacijų pasiskirstymas per visus įrašus\n")
print("labels_rec_all:\n")
print(labels_rec_all)
suma = labels_rec_all.iloc[:,1:].sum()
print('\n',' '*5, 'sum: ',str(suma.tolist())[1:-1])
total = suma.sum()
print(' '*5, 'total: ', total)


Anotacijų pasiskirstymas per visus įrašus

labels_rec_all:

    SubjCode     N    S    V  U
0      10000   720    1    0  0
1      10010   557    1    0  0
2      10020   524    0   47  0
3      10030   688  115    0  0
4      10040   961   85    0  0
5      10041   943   70    3  0
6      10050   787    0    0  0
7      10042  1115   83    0  0
8      10060   756    1    4  0
9      10043   901   83    0  0
10     10070   698    1    0  0
11     10011   597    2    1  0
12     10080   716    0   96  0
13     10031   589   74   42  0
14     10090   637    2   74  0
15     10100   645    1    0  0
16     10110   821    0    0  0
17     10021   506    0   35  0
18     10120   540    0  205  0
19     10022   528    0   40  0
20     10130   796   12    1  0

       sum:  15025, 531, 548, 0
      total:  16104


In [13]:
#  II-a dalis. SUDALIJAME PACIENTUS IR JŲ ĮRAŠUS Į TRAIN IR TEST DALIS
# Gausime df_train, df_test

from sklearn.model_selection import train_test_split
from zive_util_vu import get_annotations_distribution


print("\nII-a dalis. SUDALIJAME PACIENTUS IR JŲ ĮRAŠUS Į TRAIN IR TEST DALIS")

all_beats_attr = pd.DataFrame(columns= ['userNr', 'file_name', 'sample', 'symbol'])

# Surenkame informaciją apie pacientus
# Nuskaitomas įrašų sąrašas, suformuojamas atitinkamas dataframe df_list
file_path = Path(rec_dir, rec_list_npy)
with open(file_path,'r', encoding='UTF-8', errors = 'ignore') as f:
    data = json.loads(f.read())
df_list = pd.json_normalize(data, record_path =['data'])
print("\ndf_list:")
print(df_list)

# Susirandame anotacijų pasiskirstymą per įrašus
annot_list = ['N', 'S', 'V', 'U']
# Susikuriame pagalbinį anotacijų žodyną - dictionary beats_annot
nr_sequence = list(range(len(annot_list)))
beats_annot = dict(zip(annot_list, nr_sequence))
# print(beats_annot)

# labels_rec_all = get_annotations_distribution(df_list, rec_dir, beats_annot)
# print("\nlabels_rec_all:")
# print(labels_rec_all)
# print(labels_rec_all.dtypes)

# Susirandame anotacijų pasiskirstymą per pacientus ir pacientų skaičių
df_sum = df_list.groupby(['userId'],sort = False).sum()
# print(df_sum)

# https://sparkbyexamples.com/pandas/pandas-groupby-sum-examples/
count = df_list['userId'].value_counts()
print(f'\nViso pacientų: {len(count)}  EKG įrašų: {len(labels_rec_all)}')
print(f'\nĮrašų pasiskirstymas per pacientus')
count = count.rename("count")
frames = [df_sum, count]
result = pd.concat(frames, axis = 1)
result.index.rename ('userId', inplace= True)
# https://www.shanelynn.ie/pandas-drop-delete-dataframe-rows-columns/
print(result)

# filepath = Path(db_path, 'result.csv') 
# result.to_csv(filepath)
# print(f'Failų sąrašas ir rezultatai įrašyti į:  {filepath}')    

# Susumuojamos anotacijos
# suma = labels_rec_all.iloc[:,3:8].sum()
# print('\n',' '*18, 'sum: ',str(suma.tolist())[1:-1])
# total = suma.sum()
# print(' '*17, 'total: ', total)


# Išplečiame freimą result iki pozymio user_type, kurį naudosime kaip stratify splitinant į train, test

# Nustatome user_type pagal bendrus 'S' ir 'V' skaičius per visus paciento įrašus 
# user_type == 0, jei  bendras skaičius 'S'==0 ir bendras skaičius 'V'==0
# user_type == 1, jei  bendras skaičius 'S'> 0 ir bendras skaičius 'V'==0
# user_type == 2, jei  bendras skaičius 'S'==0 ir bendras skaičius 'V'> 0
# user_type == 3, jei  bendras skaičius 'S'> 0 ir bendras skaičius 'V'>0

result = result.reset_index()
user_types = np.zeros(len(result), dtype = int )
userIds = []
for i, row in result.iterrows():
    # print(row)
    userIds.append(row['userId'])
    if ((row['S'] == 0) & (row['V'] == 0)):
        user_types[i] = 0
    if ((row['S'] > 0) & (row['V'] == 0)):
        user_types[i] = 1
    if ((row['S'] == 0) & (row['V'] > 0)):
        user_types[i] = 2
    if ((row['S'] > 0) & (row['V'] > 0)):
        user_types[i] = 3

# print(user_types)
# result['user_type'] = user_types
# print("\nresult\n", result)

# Sudalijame duomenis į train, test

train, test = train_test_split(userIds, test_size=0.5,stratify=user_types, random_state=42)
print("\ntrain\n", train)
print("\nvalidate\n", test)




II-a dalis. SUDALIJAME PACIENTUS IR JŲ ĮRAŠUS Į TRAIN IR TEST DALIS

df_list:
    SubjCode    file_name                    userId               recordingId     N    S    V  U  incl  flag               recorded_at
0      10000  1630737.001  61632b451e325518bb017bbd  61632d471e32559827017d73   720    1    0  0     0     0  2021-09-04T06:30:01.000Z
1      10010  1630737.002  6190d4e63cd1d227c1303cea  6190d6d13cd1d2267a303e14   557    1    0  0     0     0  2021-09-04T06:30:02.000Z
2      10020  1630737.003  6144c588bd0cc52ba0275362  6145b1a6bd0cc5261f275908   524    0   47  0     0     0  2021-09-04T06:30:03.000Z
3      10030  1630737.004  61a8863e5cd547e980a4e970  61a88dd3e8f900b723c6e6ec   688  115    0  0     0     0  2021-09-04T06:30:04.000Z
4      10040  1630737.005  61632ace1e32557a62017bba  61632c1f1e32556a1c017c40   961   85    0  0     0     0  2021-09-04T06:30:05.000Z
5      10041  1630737.006  61632ace1e32557a62017bba  61632c1f1e32553353017bef   943   70    3  0     0     0  2

In [14]:

df_train = df_list.loc[df_list['userId'].isin(train)]
print("\ndf_train:")
print(df_train)

df_test = df_list.loc[df_list['userId'].isin(test)]
print("\ndf_test:")
print(df_test)




df_train:
    SubjCode    file_name                    userId               recordingId    N    S   V  U  incl  flag               recorded_at
1      10010  1630737.002  6190d4e63cd1d227c1303cea  6190d6d13cd1d2267a303e14  557    1   0  0     0     0  2021-09-04T06:30:02.000Z
2      10020  1630737.003  6144c588bd0cc52ba0275362  6145b1a6bd0cc5261f275908  524    0  47  0     0     0  2021-09-04T06:30:03.000Z
3      10030  1630737.004  61a8863e5cd547e980a4e970  61a88dd3e8f900b723c6e6ec  688  115   0  0     0     0  2021-09-04T06:30:04.000Z
6      10050  1630737.007  6144c532bd0cc54c83275360  6145ae5ebd0cc5632f2756e3  787    0   0  0     0     0  2021-09-04T06:30:07.000Z
8      10060  1630737.009  6143507abd0cc5051b275171  61448382bd0cc51834275263  756    1   4  0     0     0  2021-09-04T06:30:09.000Z
10     10070  1630737.011  61b32023cf0f347968cb1c91  61b3277dcf0f347b07cb1de0  698    1   0  0     0     0  2021-09-04T06:30:11.000Z
11     10011  1630737.012  6190d4e63cd1d227c1303cea  6190d

In [15]:
# III-a dalis. zive EKG įrašai analizuojami, formuojamas freimas all_beats_attr ir  įrašomas į diską. 

# from zive_util import print_annotations_table

print("\nEKG įrašams suformuojamas pūpsnių atributų freimas ir įrašomas į diską")

# Pacientų įrašų sąrašas bandymams:
# SubjCodes = ['10001']


# Sukūriame masyvą sekų atributų sąrašo kaupimui

train_beats_attr = pd.DataFrame(columns= ['userNr', 'recordingNr', 'sample', 'symbol'])

df = pd.DataFrame({'userNr': pd.Series(dtype='int'),
                   'recordingNr': pd.Series(dtype='int'),
                   'sample': pd.Series(dtype='int'),
                   'symbol': pd.Series(dtype='str')})


start_time = time.time()

# CIKLAS PER PACIENTŲ ĮRAŠUS
print("\nPūpsnių atributų freimas 'train':\n")

# for SubjCode in SubjCodes:
for idx, row in df_train.iterrows():    
    SubjCode = row['SubjCode']
    print("\nECG įrašas:", SubjCode)
    
    # Paciento anotacijų ir EKG įrašų nuskaitymas, sekų išpjovimas ir įrašymas, sekų atributų formavimas

    # Nuskaitome paciento anotacijas ir jų indeksus
    atr_sample, atr_symbol = read_rec_attrib(rec_dir, SubjCode)

    subject_labels = []
    beat_nr = 0
    icycle = 0

    # Ciklas per visas paciento įrašo anotacijas (simbolius) ir jų vietas (i_sample)
    for i, i_sample in enumerate(atr_sample):
    
        icycle +=1
        if (icycle%show_period == 0):
            print(icycle, end =' ') 

        # Formuojame pūpsnio atributus
        userNr, recordingNr = split_SubjCode(SubjCode)

        beats_attr = {'userNr':int(userNr), 'recordingNr':int(recordingNr), 'sample':int(i_sample), 'symbol':str(atr_symbol[i])}
        
        # Senas variantas
        # train_beats_attr = train_beats_attr.append(beats_attr, ignore_index=True)

        # Variantas su concat
        # df_new_row = pd.DataFrame([beats_attr])
        # train_beats_attr = pd.concat([train_beats_attr, df_new_row])

        # Galima perrašyti ir kitaip, naudojant pd.DataFrame.from_records
        train_beats_attr = pd.concat([train_beats_attr, pd.DataFrame.from_records([beats_attr])])

        beat_nr +=1


# Ciklo per pacientų įrašus pabaiga

# Atsikratome nepageidaujamų anotacijų: 'U'
# index_names = train_beats_attr[train_beats_attr['symbol'].isin(['U'])].index
# train_beats_attr.drop(index_names, inplace = True)
# print(train_beats_attr.info())

# Pernumeruojame indeksus, kad būtų nuo 0 iš eilės
train_beats_attr.reset_index(inplace = True, drop = True)

# Iš anotacijų suformuojame klasių numerius ir pridedame, kaip naują stulpelį
annot_labels = {key:selected_beats[value] for key, value in annot_grouping.items()}
labels_from_annot = train_beats_attr['symbol'].replace(annot_labels, inplace=False)
train_beats_attr['label'] = labels_from_annot

print("\n")
print("\nVisi duomenys\n")
labels_table, labels_sums = get_annotations_table(train_beats_attr)
print_annotations_table(labels_table, labels_sums, Flag1=True, Flag2=False)

# Įrašome sekos atributų masyvą į seq_dir aplanką
file_path = Path(rec_dir, train_beats_attr_fname)
train_beats_attr.index.name = "id"
print(train_beats_attr.dtypes)   # padaryti, kad būtų kuo talpesni
train_beats_attr.to_csv(file_path)
print("\nAtributų freimas įrašytas: į ", file_path, "\n" )

end_time = time.time()
print('\n')
runtime(end_time-start_time)

print("\nPabaiga.............")



EKG įrašams suformuojamas pūpsnių atributų freimas ir įrašomas į diską

Pūpsnių atributų freimas 'train':


ECG įrašas: 10010
100 200 300 400 500 
ECG įrašas: 10020
100 200 300 400 500 
ECG įrašas: 10030
100 200 300 400 500 600 700 800 
ECG įrašas: 10050
100 200 300 400 500 600 700 
ECG įrašas: 10060
100 200 300 400 500 600 700 
ECG įrašas: 10070
100 200 300 400 500 600 
ECG įrašas: 10011
100 200 300 400 500 600 
ECG įrašas: 10080
100 200 300 400 500 600 700 800 
ECG įrašas: 10031
100 200 300 400 500 600 700 
ECG įrašas: 10021
100 200 300 400 500 
ECG įrašas: 10022
100 200 300 400 500 


Visi duomenys

symbol        N    S    V   All
SubjCodes                      
10010       557    1    0   558
10011       597    2    1   600
10020       524    0   47   571
10021       506    0   35   541
10022       528    0   40   568
10030       688  115    0   803
10031       589   74   42   705
10050       787    0    0   787
10060       756    1    4   761
10070       698    1    0   699
10080